In [ ]:
import os
import time
import pandas as pd
import urllib.request
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
!pip install webdriver_manager

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

# 순위, 별점 크롤링

In [ ]:
import os
os.getcwd()

'C:\\Users\\82109\\Documents\\인터파크'

In [ ]:
os.chdir('C:/Users/82109')

In [ ]:
os.getcwd()

'C:\\Users\\82109'

In [ ]:
def initialize_driver():
    driver = webdriver.Chrome()
    return driver

def get_theater_links(driver, query):
    driver.get(query)
    time.sleep(1)
    number = 10

    addresslist = []

    for i in range(1, int(number) + 1):
        xpath = f"/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody/tr[{i}]/td[1]/a"
        elem = driver.find_element(By.XPATH, xpath).get_attribute("href")
        addresslist.append(elem)

    return addresslist

def scrape_reviews(driver, addresslist):
    scraped_data = []

    for lis in addresslist:
        driver.get(lis)
        time.sleep(1)

        try:
            # Scrape theater details
            title = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[1]/div/div[1]/h2").text
            elem = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/nav/div/div/ul/li[4]/a")

            dataTarget = elem.get_attribute("data-target")
            if dataTarget != "REVIEW":
                elem = driver.find_element(By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/nav/div/div/ul/li[3]/a")

            # Wait for the element to be clickable
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/nav/div/div/ul/li[4]/a")))

            elem.send_keys(Keys.ENTER)
            time.sleep(1)

            ids = []
            stars = []

            try:
                best = driver.find_element(By.CLASS_NAME, "bastBadge").text
            except:
                best = ""

            if best != "베스트":
                while True:
                    try:
                        for j in range(1, 11):
                            for i in range(1, 16):
                                a = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[3]/ul/li[" + str(i) + "]/div/div[1]/div[2]/ul/li[1]/span[1]"
                                b = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[3]/ul/li[" + str(i) + "]/div/div[1]/div[1]/div/div[1]/div"
                                id = driver.find_element(By.XPATH, a).text
                                star = driver.find_element(By.XPATH, b).get_attribute("data-star")
                                ids.append(id)
                                stars.append(star)

                            # 다음페이지 클릭
                            if j < 10:
                                botton = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[3]/div[2]/ol/li[" + str(j + 1) + "]/a"
                                elem = driver.find_element(By.XPATH, botton)
                                elem.send_keys(Keys.ENTER)
                                time.sleep(1)
                            else:
                                # 10번 다음페이지
                                elem = driver.find_element(By.CLASS_NAME, "pageNextBtn.pageArrow")
                                elem.send_keys(Keys.ENTER)
                                time.sleep(1)
                    except:
                        break

            # 베스트리뷰가 있으면 실행
            else:
                while True:
                    try:
                        for j in range(1, 11):
                            for i in range(1, 16):
                                a = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[4]/ul/li[" + str(i) + "]/div/div[1]/div[2]/ul/li[1]/span[1]"
                                b = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[4]/ul/li[" + str(i) + "]/div/div[1]/div[1]/div/div[1]/div"
                                id = driver.find_element(By.XPATH, a).text
                                star = driver.find_element(By.XPATH, b).get_attribute("data-star")
                                ids.append(id)
                                stars.append(star)

                            # 다음페이지 클릭
                            if j < 10:
                                botton = "/html/body/div[1]/div[5]/div[1]/div[2]/div[2]/div/div/div[4]/div[2]/ol/li[" + str(j + 1) + "]/a"
                                elem = driver.find_element(By.XPATH, botton)
                                elem.send_keys(Keys.ENTER)
                                time.sleep(1)
                            else:
                                # 10번 다음페이지
                                elem = driver.find_element_by_class_name("pageNextBtn.pageArrow")
                                elem.send_keys(Keys.ENTER)
                                time.sleep(1)
                    except:
                        break

            # Append scraped data for this theater
            scraped_data.append({'Title': title, 'IDs': ids, 'Stars': stars})

        except Exception as e:
            print(f"Error while processing {title}: {str(e)}")

    return scraped_data

def save_to_csv(data_list, save_path):
    if data_list:
        combined_data = pd.DataFrame(data_list)
        combined_data.to_csv(save_path, index=False, encoding='cp949')

current_directory = os.getcwd()
save_path = os.path.join(current_directory, "연극.csv")

driver = initialize_driver()
query = "http://ticket.interpark.com/TPGoodsList.asp?Ca=Dra"
addresslist = get_theater_links(driver, query)
scraped_data = scrape_reviews(driver, addresslist)
save_to_csv(scraped_data, save_path)
driver.quit()

# 리뷰 크롤링

In [ ]:
driver = webdriver.Chrome()
driver.get("https://tickets.interpark.com/goods/18011275")
driver.implicitly_wait(5)
time.sleep(2)
nav = driver.find_elements(By.CLASS_NAME, "navItem")
for i in nav:
     print(i.text)

공연정보
캐스팅정보
판매정보
관람후기999+
기대평215
Q&A182


In [ ]:
driver = webdriver.Chrome()
driver.get("https://tickets.interpark.com/goods/23011234")
driver.implicitly_wait(5)
time.sleep(2)
# 팝업창 닫기
driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn").click()
nav = driver.find_elements(By.CLASS_NAME, "navItem")

for i in nav:
    if "관람후기" in i.text:
        i.click()
        time.sleep(1)

total = []

for i in driver.find_elements(By.CLASS_NAME, "bbsItem"):
    title = i.find_element(By.CLASS_NAME, "bbsTitle").text
    content = i.find_element(By.CLASS_NAME, "bbsText").text
    total.append([title, content])

print(total)

[['"그런 소리가 났다"', '페이지 펄럭 넘어가는 소리\n\n비밀기지가 "드디어" 펼쳐낸 새로운\n\n코ㅏO'], ['연어, 인생의 바다 속을 가로지르다', '삶이란 바다와 같다. 드러나는 파도도 거세지만 깊은 물 속, 보이지 않는 해류도 그못지 않다. 인생도 마찬가지다. 겉으로 드러나 보이는 삶의 문제 못지 않게 내면 안에 해결되어야 할 어려움들 또한 많다.\n\n내면과 외면 중 삶의 문제가 어느 게 더 큰지 따지는 것은 불필요한 논쟁이다. 하지만, 연어가 견뎌낸 위로부터 내려오는 해류의 압박감은 그 어느 것에 뒤지지 않음을 자신한다. 작가가 작품을 통해 말하고자 했던 것은 이 내면 안의 해류를 매일 거슬러 올라가는 우리 모습이지 않았을까.\n\n분투하는 삶 속의 의미를 찾는다는게 누군가에게는 허공에 떠도는 꽹과리 같을 수 있겠다. 하지만, 그 의미를 너무 먼 곳이 아닌 우리 가까이에서 찾아 보는 것은 어떨까. 작품 연어는 해류를 거슬러 올라갈 수 있는 힘이 그리 멀리 있지 않음을 암시한다. 마주한 삶의 문제 속에서 내면의 해류를 이겨내고자 하는 모든 이들에게 이 작품을 추천한다.'], ['나와 대화할 수 있던 시간', '‘청소년’ 키워드라고 해서 가볍게 보려다가\n내면 속 어린 나와 불안정했던 나를 만날 수 있습니다.\n인물 한명한명 저를 투영해볼 수 있었습니다.\n\n연어랑 용서의신은 잔잔하지만 많은 고민을 하게 해주었고, 쾅은 말그대로 생각지 못하게 어린 나를 발견하고 공감받게 되어 가장 인상깊었습니다. (저만 중간에 눈물 흘린 줄 알았는데 다른 분 후기도 ㅎㅎ)\n\n음향 조명 화면에 배우들의 연기에 시간 가는 줄 모르고 봤습니다.\n특히 쾅은 한번더 보고싶네요 !'], ['연어 & 용서의 신', '아직은 성장 중이고, 미숙한 시기를 지나고 있는\n청소년들을 참신한 시선으로\n바라볼 수 있었던 연극이었습니다.\n\n연출 방식, 배우들의 열연,\n그리고 대사 한마디 한마디가\n극의 분위기와 인물을 잘 전달해주어\n관객 입장에서 몰입하기 좋았습니다.\

In [ ]:
driver = webdriver.Chrome()
driver.get("https://tickets.interpark.com/goods/P0003343")
driver.implicitly_wait(5)
time.sleep(2)
# 팝업창 닫기
try:
    close_button = driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn")
    close_button.click()
    time.sleep(0.5)
except NoSuchElementException:
    pass
nav = driver.find_elements(By.CLASS_NAME, "navItem")

for i in nav:
    if "관람후기" in i.text:
        i.click()
        time.sleep(1)

total = []

for i in driver.find_elements(By.CLASS_NAME, "bbsItem"):
    title = i.find_element(By.CLASS_NAME, "bbsTitle").text
    content = i.find_element(By.CLASS_NAME, "bbsText").text
    total.append([title, content])

print(total)

[['카르멘', '카르멘과 돈 호세 두 인물의 충돌이 잘 담겨있는 작품입니다! 저는 한번 더 보고 싶은 작품이네요'], ['미카엘라의 사랑', '미카엘라는 끝까지 바보같은 사람이었다.\n돈 호세는 싸이코패스 같은 사람이었고,\n카르멘은 자신의 방탕한 생활을 자유로움이라 포장하는 사람이었다.\n\n이 셋중에 하나로 살아가야 한다면, 미카엘라 같은 사랑을 하며 살아가고 싶다. 그나마 이들 중에서 가장 진정한 사랑을 하는 사람인 것 같아서..\n\n또 보고싶다, 미카엘라의 사랑. 미카엘라의 포효.'], ['카르멘에 홀딱 반함!', '연기를 다 너무 잘하셔서 놀랐네요\n역시 연극 배우분들은 발성부터가 다른 것 같아요.\n특히 주인공 두분 마지막 장면까지 진짜 열연!!\n마지막에 (스포 금지라서) 보면서 좀 울엇네요.\n서지우 배우는 마스크가 진짜 매력적이시라 반햇어요!\n여튼 스페인 세비야에서 플라멩고 봤던 추억이 떠올라서 좋았습니다.\n이런 공연은 두번 세번 봐도 안아까워요!!'], ['21세기의 카르멘', '(스포o)\n\n팜므파탈의 대명사와 같은 카르멘을, 지금 이 시대에 공연한다는 점에서 궁금해서 못 참고 보러 갔음.\n요약하자면 우리가 아는 그 카르멘이라고 할 수는 없음. 그러나 ‘팜므파탈의 대명사’니 ‘매력적인 여성’이니 하는 수식어를 기대하고 간 것 가체가 나 스스로 또한 프레임 풀장착 하고 갔던 것은 아닌지 반성함. ‘발에 맞지 않는 구두는 싫어’라고 우는 여자에게 나역시 ‘카르멘이 좀 다르네?‘하고 코르셋 꽉 조이고 있었던 거임. 그런 점에서 꿏나고 곱씹을수록 발견할만한 의미가 많았던 듯함.\n비슷한 맥락에서 돈 호세도 여자에게 버려진 순애보의 비극적인 사랑... 절대 아니고 스토킹 범죄자ㅇㅇ... 요즘 흉흉한 사회상을 적극 반영한 작품이 아닌가?\n\n맥락도 맥락인데 배우들 춤추는 거랑 무대랑 음향 다 꿀잼이니 꼭 보시길 추천'], ['매력없음', '음악만 들어도 그 매력이 물씬 묻어났던 카르멘인데 매력적이지 않았음. 돈호세역시 그녀에게 반해 사랑을

In [ ]:
total = pd.DataFrame(total, columns=['제목', '리뷰'])
total.head()

,제목,리뷰
0,카르멘,카르멘과 돈 호세 두 인물의 충돌이 잘 담겨있는 작품입니다! 저는 한번 더 보고 싶...
1,미카엘라의 사랑,"미카엘라는 끝까지 바보같은 사람이었다.\n돈 호세는 싸이코패스 같은 사람이었고,\n..."
2,카르멘에 홀딱 반함!,연기를 다 너무 잘하셔서 놀랐네요\n역시 연극 배우분들은 발성부터가 다른 것 같아요...
3,21세기의 카르멘,"(스포o)\n\n팜므파탈의 대명사와 같은 카르멘을, 지금 이 시대에 공연한다는 점에..."
4,매력없음,음악만 들어도 그 매력이 물씬 묻어났던 카르멘인데 매력적이지 않았음. 돈호세역시 그...


In [ ]:
driver = webdriver.Chrome()
driver.get("https://tickets.interpark.com/goods/P0003343")
driver.implicitly_wait(5)
time.sleep(2)
## 팝업창이 뜨면 닫기
try:
    close_button = driver.find_element(By.CLASS_NAME, "popupCloseBtn.is-bottomBtn")
    close_button.click()
    time.sleep(1)
except NoSuchElementException:
    pass

nav = driver.find_element(By.CLASS_NAME, "navItem")
if "관람후기" in nav.text:
    nav.click()
    time.sleep(2)

total = []

while True:
    pages = driver.find_element(By.CLASS_NAME, "pageNumWrap").find_elements(By.TAG_NAME, "li")
    for i in pages:
        i.click()
        time.sleep(2)

    for i in driver.find_elements(By.CLASS_NAME, "bbsItem"):
        title = i.find_element(By.CLASS_NAME, "bbsTitle").text
        content = i.find_element(By.CLASS_NAME, "bbsText").text
        total.append([title, content])

    print(f"{len(total)}개의 리뷰를 저장했습니다.")

    try:
        driver.find_element(By.CLASS_NAME, "pageNextBtn.pageArrow").click()
    except:
        break

driver.quit()

AttributeError: 'list' object has no attribute 'find_elements'